In [1]:
import pickle
import numpy as np
from dispatch_jobs import DB

KEY_PREFIX = 'gbrph_'
keys = DB.keys(KEY_PREFIX + '*')

# For testing
# keys = ['gbrph_' + p for p in ['1a1e', '1a4r']]

keys = list(filter(lambda x: DB.hgetall(x)['finished'] == 'True' and DB.hgetall(x)['error'] == 'False', keys))

INFO:root:Database connection successful


In [2]:
print(f'Number of keys: {len(keys)}')

Number of keys: 5190


In [3]:
from gudhi.representations.vector_methods import PersistenceImage

def diagram_to_image(diagram):
    pim = PersistenceImage(bandwidth=0.2355, resolution=[100,100], im_range=[0, 50, 0, 50])

    # check that the third column is indeed only 0, 1, 2
    # assert (lambda unique_elements_in_array=np.unique(diagram[0, :, 2])
                # : np.array_equal(unique_elements_in_array, np.array([0, 1, 2])))()

    # Now reshape into (3, n, 2)
    def get_nd_diagram(diagram, dim):
        filtered_diagrams = list(filter(lambda x: x[2] == dim, diagram[0, :, :]))
        return np.array(filtered_diagrams)[:, :2]

    gudhi_format_diagrams = [get_nd_diagram(diagram, 0), get_nd_diagram(diagram, 1), get_nd_diagram(diagram, 2)]


    diagrams_clipped = [np.clip(diagram, 0, 50) for diagram in gudhi_format_diagrams]
    imgs = pim.fit_transform(diagrams_clipped)

    return imgs

In [4]:
from tqdm import tqdm
import math
from pathlib import Path
import glob
from multiprocessing import cpu_count, Pool
from tqdm.contrib.concurrent import process_map

num_datapoints_per_file = 500

Path('persistence_images').mkdir(parents=True, exist_ok=True)

for file_index in range(math.ceil(len(keys) / num_datapoints_per_file)):
    print(f'File index: {file_index}')
    # To-Do: Migrate this section

    observations = []
    binding_affinities = []

    # for key in tqdm(keys[file_index * num_datapoints_per_file: (file_index + 1) * num_datapoints_per_file]):

    def process(key):
        info = DB.hgetall(key)
        if info['finished'] == 'True' and info['error'] == 'False':
            save_file = info['save_file']

            with open(save_file, 'rb') as f:
                d = pickle.load(f)

            pw_opposition_diagrams = d['pw_opposition_diagrams']  # list of 36 diagrams, each of shape (1, n, 3)
            other_persistence_diagrams = d['other_persistence_diagrams']  # other_persistence_diagrams: list of 4 diagrams, each of shape (1, n, 3)

            all_diagrams = pw_opposition_diagrams + other_persistence_diagrams

            all_images = list(map(diagram_to_image, all_diagrams))

            # observations.append(np.array(all_images))

            # binding_affinities.append(float(info['-logKd/Ki']))
            print('fin')

            return np.array(all_images), float(info['-logKd/Ki'])

    # for key in tqdm(keys[file_index * num_datapoints_per_file: (file_index + 1) * num_datapoints_per_file]):

    # observations, binding_affinities = zip(*process_map(process, keys[file_index * num_datapoints_per_file: (file_index + 1) * num_datapoints_per_file], max_workers=8))

    with Pool(4) as p:
        results = p.map(process, keys[file_index * num_datapoints_per_file: (file_index + 1) * num_datapoints_per_file])

    observations, binding_affinities = zip(*results)

    observations = np.array(observations)
    binding_affinities = np.array(binding_affinities)

    print(observations.shape)
    print(binding_affinities.shape)

    with open(f'persistence_images/observations_{file_index}.npy', 'wb') as f:
        np.save(f, observations)

    with open(f'persistence_images/binding_affinities_{file_index}.npy', 'wb') as f:
        np.save(f, binding_affinities)


# # Loading the files back in
# observations = []
# binding_affinities = []

# for file_index in range(len(list(Path('persistence_images').glob('binding_affinities_*.npy')))):
#     with open(f'npys/observations_{file_index}.npy', 'rb') as f:
#         observations.append(np.load(f))

#     with open(f'npys/binding_affinities_{file_index}.npy', 'rb') as f:
#         binding_affinities.append(np.load(f))


# observations = np.concatenate(observations)
# binding_affinities = np.concatenate(binding_affinities)

# print(observations.shape)
# print(binding_affinities.shape)



File index: 0


fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
fin
